<a href="https://colab.research.google.com/github/viksingh/pythonfinance/blob/master/100dayhighstrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import time
import pdb
start_time = time.time()


#from google.colab import drive
#drive.mount('/content/drive')
#!{sys.executable} -m pip install PyDrive
#!{sys.executable} -m pip install yfinance


import yfinance as yf

# Check if index is going up

MONEYFOREACHSTOCK = 2500
AMOUNTSPENT = 0
AMOUNTEARNERD = 0
SELLPRICE = 0
STOPLOSSFACTORINCREASINGINDEX = 0.80
STOPLOSSFACTORREDUCINGINDEX = 0.90
NUMBEROFSTOCKS = 0
CURRENTAMOUNT = 0
TOTALSTOCKSHELD = 0

TOTALSPENT = 0
TOTALVALUE = 0
TOTALSTOPLOSSMONEY = 0
INDEXPATH = '/content/drive/My Drive/Colab Notebooks/data/index2.csv'
STOCKCOMBINEDPATH = '/content/drive/My Drive/Colab Notebooks/data/stockscombined.csv'

dfindex = pd.read_csv(INDEXPATH, header=0)
dfstocks = pd.read_csv(STOCKCOMBINEDPATH, header=0 )

# Create a dataframe to store currently bought stocks
# Fields : STOCK, BOUGHT DATE , BOUGHT PRICE , QTY, UPDATE DATE, LAST PRICE, RoC % , BOUGHTPRICE , UPDTPRICE
# May be add an entry for each week when this gets checked
print('READ BOTH FILES')



from datetime import date, timedelta

start_date = date(2019, 1, 11)
end_date = date(2019, 3, 8)
delta = timedelta(days=7)
dfStockUniqueList = dfstocks
#dfStockUniqueList[1].unique()
#dfStockUniqueList['STOCK'] = dfStockUniqueList.groupby('STOCK')['STOCK'].rank( ascending=False)
companiesList = dfStockUniqueList.STOCK.unique()
print(companiesList)

dfStocksCurrentlyBought = pd.DataFrame(columns=['STOCK','PURCHASEDATE','PURCHASEPRICE','QTY','UPDATEDATE','UPDATEPRICE','MAXPRICE','ROC', 'ACTIVE'])

while start_date <= end_date:
# Check if index is increasing / decreasing this week
    StartDate = start_date.strftime("%Y-%m-%d")
    queryIndexWithDate = 'DATE' + '==' + "'" + StartDate + "'"
    dfNasdaqCheckIndexIncreasing = dfindex.query(queryIndexWithDate)
    if dfNasdaqCheckIndexIncreasing.size > 0:
        isIncreasing = dfNasdaqCheckIndexIncreasing['CROSS'].iloc[-1]
        print('Index : ' + str(StartDate) + ' - ' + str(isIncreasing))

        # check if any stock needs to be sold in the current week
        # loop over stocks currently bought

# ######## SELL STOCK LOGIC ###################################################
# ######## SELL STOCK LOGIC ###################################################

        dfstocksActive = dfStocksCurrentlyBought.query("ACTIVE == 'X'")
        stockBoughtList = dfstocksActive.STOCK.unique()

        if( isIncreasing ==1 ):
            for stockToSellCheck in stockBoughtList:
#                print('CHECK IF WE SHOULD SELL ')

#                queryDateNasDaqThisWeek = 'Date' + '==' + "'" + CurrentWeekDate + "'"
                queryGetCurrentPriceWithDate = 'DATE' + '==' + "'" + StartDate + "'"
                dfStockWithCurrentDate = dfstocks.query(queryGetCurrentPriceWithDate)
                queryGetCurrentPriceWithDateSTOCK = 'STOCK' + '==' + "'" + stockToSellCheck + "'"
                dfStockWithCurrentDate = dfStockWithCurrentDate.query(queryGetCurrentPriceWithDateSTOCK)
#                print(dfStockWithCurrentDate)
# SELLING RULES
# If increasing index this week : current price is higher than 0.8 MAX PRICE which will be the last row of
# If currennt price

                MAXPRICEFORSTOCKLIST = dfStocksCurrentlyBought.groupby(['STOCK'])['MAXPRICE'].max()
                stockPriceThisStock = float (MAXPRICEFORSTOCKLIST.get(key=stockToSellCheck))

                # Get current price if it's less than 0.8 MAX PRICE if index increasing or less than 0.9 index decreasing
                if dfStockWithCurrentDate.size > 0:
                    CURRENTSTOCKPRICE = dfStockWithCurrentDate.iloc[-1]['PRICE']

                    dfCurrentStockData = dfStocksCurrentlyBought.query(queryGetCurrentPriceWithDateSTOCK )
                    STOCKQTY = dfCurrentStockData.iloc[-1]['QTY']

                    if (isIncreasing ==1):
                        STOPLOSSPRICE = STOPLOSSFACTORINCREASINGINDEX *  stockPriceThisStock
                    else:
                        STOPLOSSPRICE = STOPLOSSFACTORREDUCINGINDEX *  stockPriceThisStock



                    if (CURRENTSTOCKPRICE < STOPLOSSPRICE):
                        dfStocksCurrentlyBought.loc[ (dfStocksCurrentlyBought.STOCK == stockToSellCheck  ),'ACTIVE'] = ''
                        print("SELL : STOCK VALUE IS : " + str(stockToSellCheck) +' - ' + str(StartDate) +' - ' +  ' Total Amount : ' + str(CURRENTSTOCKPRICE * STOCKQTY ))
                        TOTALSTOCKSHELD = TOTALSTOCKSHELD - 1
#                        print('Number of stocks reduced to ' + str(TOTALSTOCKSHELD))
                        TOTALSTOPLOSSMONEY = TOTALSTOPLOSSMONEY + CURRENTSTOCKPRICE * STOCKQTY
                        TOTALVALUE = TOTALVALUE + TOTALSTOPLOSSMONEY
#                        print('Total stop loss money : ' + str(TOTALSTOPLOSSMONEY))

#                    else:
#                        print("STOCK VALUE IS : " + str(stockToSellCheck) +' - ' + str(CURRENTSTOCKPRICE) + ' - '+ str(StartDate) +' - ' +  ' Total Amount : ' + str(CURRENTSTOCKPRICE * STOCKQTY ))


# **************** IDENTIFY STOCKS TO BUY ***********************************
# **************** IDENTIFY STOCKS TO BUY ***********************************

        if (isIncreasing == 1):
            for  stock in companiesList :

                queryGetCurrentStockData = 'STOCK' + '==' + "'" + stock + "'"
                dfCurrentStock = dfstocks.query(queryGetCurrentStockData)

# Lookback 100 days to see if stock is really breaking out
                PERIODTOLOOKBACKFORSTOCKS = 20 # 20 weeks
                queryDateForCURRENTSTOCK = "DATE" + "<" + "'" + StartDate + "'"
                dfCurrentStocklookBackData = dfCurrentStock.query(queryDateForCURRENTSTOCK)
                dfCurrentStocklookBackData = dfCurrentStocklookBackData.tail(PERIODTOLOOKBACKFORSTOCKS)

# CHeck if we have enough data
                if dfCurrentStocklookBackData.shape[0] > PERIODTOLOOKBACKFORSTOCKS - 1:
                    start = 0
                    length = PERIODTOLOOKBACKFORSTOCKS - 1
                    stockHighestPointValid = True

                    latestpriceIndex = length - start
                    for n in range(start, start + length):
                        currentweekPrice  = dfCurrentStocklookBackData['PRICE'].iloc[latestpriceIndex]
                        lastweekPrice = dfCurrentStocklookBackData['PRICE'].iloc[n]
                        if (lastweekPrice > currentweekPrice):
                            stockHighestPointValid = False
                            break




                    if( stockHighestPointValid == True ):
                        STOCK = dfCurrentStocklookBackData['STOCK'].iloc[-1]
                        VOLUME = dfCurrentStocklookBackData['VOLUME'].iloc[-1]
                        DATE = StartDate #dfCurrentStocklookBackData['DATE'].iloc[-1]
                        ROI = dfCurrentStocklookBackData['ROC100'].iloc[-1]
## how to use this .it should be the same as last price
                        MAXPRICE = dfCurrentStocklookBackData.loc[dfCurrentStocklookBackData['PRICE'].idxmax()]['PRICE']

                        if (ROI > 50 and VOLUME > 1000000 and MAXPRICE > 5 and stockHighestPointValid == True):

                            dfCheckStockALreadyAdded = dfStocksCurrentlyBought.query(queryGetCurrentStockData)
                            if dfCheckStockALreadyAdded.size == 0:
                                TOTALSTOCKSHELD = TOTALSTOCKSHELD + 1
#                                print('Number of stocks increased to ' + str(TOTALSTOCKSHELD))
                                CURRENTPRICESTOCKDATE = ''
                                NUMBEROFSTOCKS = int(MONEYFOREACHSTOCK / MAXPRICE)
                                AMOUNTSPENT = NUMBEROFSTOCKS *  MAXPRICE
#                                print(STOCK + ',' + str(MAXPRICE) + ',' + str(ROI) + ',' + str(VOLUME) + ',' + str(DATE) + ',' +  str(NUMBEROFSTOCKS))
                                CURRPRICEYAHOOFIN = str(yf.Ticker(STOCK).info['regularMarketPrice'])
                                CURRVALUEYAHOOFIN = str( yf.Ticker(STOCK).info['regularMarketPrice'] * NUMBEROFSTOCKS)
                                print( str(TOTALSTOCKSHELD) +' Buy : ' + STOCK + ' - ' + str(StartDate) + ' @ ' + str(MAXPRICE) + ' Curr Price : '+ CURRPRICEYAHOOFIN  + ' - ' + str(AMOUNTSPENT) +' - '+ 'Curr Value : '+ CURRVALUEYAHOOFIN)
                                dfStocksCurrentlyBought = dfStocksCurrentlyBought.append({'STOCK':STOCK , 'PURCHASEDATE' : DATE , 'PURCHASEPRICE': MAXPRICE , 'QTY': NUMBEROFSTOCKS , 'UPDATEDATE': DATE ,'UPDATEPRICE': MAXPRICE , 'MAXPRICE' : MAXPRICE, 'ROC': ROI , 'ACTIVE': 'X' }, ignore_index=True)
                                TOTALSPENT = TOTALSPENT + AMOUNTSPENT


#                print(dfCurrentStock)












        else:
            print('NO RECOMMENDATIONS AS INDEX IS GOING DOWN : ' + str(start_date))



    start_date += delta




stockCurrtList = dfStocksCurrentlyBought.query("ACTIVE == 'X'").STOCK.unique()

print('SIZE : ' + str(stockCurrtList.size))
for stockCurrValue in stockCurrtList:
    queryGetCurrentStockDataCurrValue = 'STOCK' + '==' + "'" + stockCurrValue + "'"
    dfCurrentStockCurrValue = dfstocks.query(queryGetCurrentStockDataCurrValue)

#    queryGetCurrentPriceWithDateCurrValue = "DATE" + "==" + "'" + EndDate + "'"
#    dfCurrentStockCurrValue = dfCurrentStockCurrValue.query(queryGetCurrentPriceWithDateCurrValue)
    dfCurrentStockCurrValue = dfCurrentStockCurrValue.tail(1)


# SUMMARY
    
    if dfCurrentStockCurrValue.shape[0] > 0:
#                PRICEFORSUMMARY = float(dfCurrentStockCurrValue['PRICE'].iloc[0])
        PRICEFORSUMMARY = float(yf.Ticker(stockCurrValue).info['regularMarketPrice'])
        dfCurrentStockQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataCurrValue)['QTY'].iloc[0]
        STOCKVALUE = float(PRICEFORSUMMARY * dfCurrentStockQty)
        print('Value : ' + stockCurrValue + ' - ' + str(PRICEFORSUMMARY) + '- ' + str(dfCurrentStockQty) + ' - ' + str(STOCKVALUE))
        TOTALVALUE = TOTALVALUE + STOCKVALUE
        


print('AMOUNT SPENT : ' + str(TOTALSPENT) + ' CURR VALUE: ' + str(TOTALVALUE) + ' ROI % = ' + str( (TOTALVALUE - TOTALSPENT)/ TOTALVALUE * 100 ) + '%' )






READ BOTH FILES
['opnt' 'flic' 'grvy' ... 'mik' 'xpel' 'entx']
Index : 2019-01-11 - 0
NO RECOMMENDATIONS AS INDEX IS GOING DOWN : 2019-01-11
Index : 2019-01-18 - 1
1 Buy : wday - 2019-01-18 @ 167.46 Curr Price : 169.51 - 2344.44 - Curr Value : 2373.14
2 Buy : qnst - 2019-01-18 @ 18.36 Curr Price : 15.29 - 2496.96 - Curr Value : 2079.44
3 Buy : gluu - 2019-01-18 @ 9.38 Curr Price : 6.39 - 2495.0800000000004 - Curr Value : 1699.74
